In [3]:
import requests
import json
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor

def fetch_data(url):
    response = requests.get(url, auth=(username, password))
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao fazer a solicitação à API: {response.status_code}")
        return None

def search_value_in_json(input_value, json_data, form_id):
    if not json_data:
        return

    for step in json_data.get('processingSteps', []):
        condition = step.get('condition', {})
        if 'conditionalFieldCriteria' in condition:
            for criteria in condition['conditionalFieldCriteria']:
                criteria_condition = criteria.get('condition', {})
                if 'value' in criteria_condition:
                    criteria_value = criteria_condition['value']
                    if input_value.lower() == criteria_value.lower():
                        print("Valor encontrado:", criteria_value, "no step com ID", form_id )

def process_forms(forms):
    with ThreadPoolExecutor() as executor:
        for form in forms:
            executor.submit(process_form, form)

def process_form(form):
    form_id = form['id']
    print(f"Processando formulário com ID {form_id}")
    form_url = f"{base_url}/api/REST/2.0/assets/form/{form_id}"
    form_data = fetch_data(form_url)
    if form_data:
        search_value_in_json(search_value, form_data, form_id)

if __name__ == "__main__":
    load_dotenv()

    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")

    base_url = "https://secure.p04.eloqua.com"
    
    api_url_forms = f"{base_url}/api/REST/2.0/assets/forms"

    search_value = input("Digite o valor a ser pesquisado: ")

    forms_data = fetch_data(api_url_forms)
    total_pages = forms_data['total'] // forms_data['pageSize'] + 1

    for page in range(1, total_pages + 1):
        page_url = f"{api_url_forms}?page={page}&pageSize=1000"
        forms_data = fetch_data(page_url)

        if forms_data:
            process_forms(forms_data['elements'])


Processando formulário com ID 5992
Processando formulário com ID 5384
Processando formulário com ID 3903
Processando formulário com ID 1818
Processando formulário com ID 5790
Processando formulário com ID 7928
Processando formulário com ID 6021
Processando formulário com ID 4084
Processando formulário com ID 8554
Processando formulário com ID 3673
Processando formulário com ID 5842
Processando formulário com ID 7271
Processando formulário com ID 4799Processando formulário com ID 8418

Processando formulário com ID 6343
Processando formulário com ID 4804
Processando formulário com ID 5319
Processando formulário com ID 1404Processando formulário com ID 5977

Processando formulário com ID 7403
Processando formulário com ID 6583
Processando formulário com ID 8015
Processando formulário com ID 6015
Processando formulário com ID 4710
Processando formulário com ID 3766
Processando formulário com ID 8185
Processando formulário com ID 8066
Processando formulário com ID 5719
Processando formulár

In [2]:
import os
from dotenv import load_dotenv
import requests
from concurrent.futures import ThreadPoolExecutor

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter variáveis de ambiente
username = os.getenv("ELOQUA_USERNAME")
password = os.getenv("ELOQUA_PASSWORD")
base_url = "https://secure.p04.eloqua.com"

def get_active_segments():
    active_segments = []
    page = 1
    page_size = 1000 

    while True:
        url = f"{base_url}/api/REST/2.0/assets/contact/segments?page={page}"
        print(f"Obtendo segmentos da página {page}")
        response = requests.get(url, auth=(username, password))
        data = response.json()
        active_segments.extend([segment['name'] for segment in data['elements'] if segment['currentStatus'] == 'Active'])
        
        total_segments = data['total']
        processed_segments = page * page_size
        if processed_segments >= total_segments:
            break  

        page += 1

    return active_segments


def check_segment_name(name, depth='complete'):
    active_segments = get_active_segments()
    print(len(active_segments))
    def check_segment(segment_name):
        url = f"{base_url}/api/REST/2.0/assets/contact/segments?search='{segment_name}'&depth={depth}"
        response = requests.get(url, auth=(username, password))
        data = response.json()
        if data['total'] > 0:
            for element in data['elements']:
                if 'elements' in element:
                    for sub_element in element['elements']:
                        if 'criteria' in sub_element['filter']:
                            for criterion in sub_element['filter']['criteria']:
                                criterion_value = criterion['condition']['value']
                                print(f"Verificando o segmento {segment_name} com valor do critério: {criterion_value}")
                                if name.lower() == criterion_value.lower():
                                    print(f"O valor {name} foi encontrado no segmento {segment_name}")
                                    


    with ThreadPoolExecutor() as executor:
        executor.map(check_segment, active_segments)



user_input = input("Digite o nome: ")
if check_segment_name(user_input):
    print(f"O nome '{user_input}' está presente nos segmentos ativos.")


Obtendo segmentos da página 1
Obtendo segmentos da página 2
Obtendo segmentos da página 3
Obtendo segmentos da página 4
Obtendo segmentos da página 5
Obtendo segmentos da página 6
Obtendo segmentos da página 7
Obtendo segmentos da página 8
Obtendo segmentos da página 9
Obtendo segmentos da página 10
Obtendo segmentos da página 11
Obtendo segmentos da página 12
Obtendo segmentos da página 13
Obtendo segmentos da página 14
Obtendo segmentos da página 15
Obtendo segmentos da página 16
2290
Verificando o segmento PROD PL Finance Weekly Update com valor do critério: bakermckenzie.com
Verificando o segmento GLI Converted Leads com valor do critério: Sales Qualified
Verificando o segmento GLI Disqualified Leads com valor do critério: Disqualified
Verificando o segmento GLI Disqualified Leads com valor do critério: Discarded
Verificando o segmento GLI Disqualified Leads com valor do critério: DIs
Verificando o segmento Consent Management - 3rd Party Consent com valor do critério: 3rd Party
Ver

In [ ]:
# Script para verificar se um valor está presente em alguma campanha ativa



In [1]:
from flask import Flask, jsonify, request
import os
import requests
from dotenv import load_dotenv

app = Flask(__name__)
load_dotenv()

username = os.getenv("ELOQUA_USERNAME")
password = os.getenv("ELOQUA_PASSWORD")

base_url = "https://secure.p04.eloqua.com"

auth = (username, password)

# --------------------------------------------------------------------------

def search_in_json(json_obj, search_term):
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            if key == "name" and search_term.lower() in value.lower():
                return True
            elif isinstance(value, (dict, list)):
                if search_in_json(value, search_term):
                    return True
    elif isinstance(json_obj, list):
        for item in json_obj:
            if search_in_json(item, search_term):
                return True
    return False

@app.route('/search_programs', methods=['GET'])
def search_programs():
    programs_url = f"{base_url}/api/rest/2.0/assets/programs?search=currentStatus='active'"
    program_details_url = f"{base_url}/api/rest/2.0/assets/program/{{}}"
    
    search_name = request.args.get('name', '')  # Pega o parâmetro de busca da URL

    response = requests.get(programs_url, auth=auth)
    results = []

    if response.status_code == 200:
        data = response.json()

        for element in data["elements"]:
            program_id = element["id"]
            specific_program_url = program_details_url.format(program_id)

            try:
                response = requests.get(specific_program_url, auth=auth)

                if response.status_code == 200:
                    program_details = response.json()

                    for element in program_details["elements"]:
                        if search_in_json(element, search_name):
                            name = element["name"]
                            member_count = element["memberCount"]
                            results.append({
                                'Program ID': program_id,
                                'Element Name': name,
                                'Member Count': member_count
                            })

            except Exception as e:
                return jsonify({'error': f"Erro ao processar programa ID {program_id}: {str(e)}"}), 500

        return jsonify(results)

    else:
        return jsonify({'error': f"Erro ao fazer a requisição: {response.status_code}"}), response.status_code



# --------------------------------------------------------------------------


if __name__ == '__main__':
    app.run(debug=True)                                

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\6125974\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
